In [30]:
!nvidia-smi

Sat Aug  9 23:23:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             31W /  250W |    3025MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [31]:
!nvidia-smi topo -m

	GPU0	CPU Affinity	NUMA Affinity	GPU NUMA ID
GPU0	 X 	0-3	0		N/A

Legend:

  X    = Self
  SYS  = Connection traversing PCIe as well as the SMP interconnect between NUMA nodes (e.g., QPI/UPI)
  NODE = Connection traversing PCIe as well as the interconnect between PCIe Host Bridges within a NUMA node
  PHB  = Connection traversing PCIe as well as a PCIe Host Bridge (typically the CPU)
  PXB  = Connection traversing multiple PCIe bridges (without traversing the PCIe Host Bridge)
  PIX  = Connection traversing at most a single PCIe bridge
  NV#  = Connection traversing a bonded set of # NVLinks


In [1]:
!pip install transformers datasets evaluate huggingface_hub wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 661.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64"

In [1]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
import wandb

u=UserSecretsClient()
login(u.get_secret("HF"))
# print(u.get_secret("HF"))
wandb.login(key=u.get_secret("WB"))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sp4ss (sp4ss-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
#@Title: Training
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

2025-08-09 23:00:51.917849: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754780452.116785      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754780452.171353      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
dataset = load_dataset("DanielHesslow/SwissProt-Pfam")
train_ds = dataset["train"].train_test_split(test_size=0.1, seed=42)
val_ds = train_ds["test"]
train_ds = train_ds["train"]

README.md:   0%|          | 0.00/494 [00:00<?, ?B/s]

dataset_infos.json: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/149M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/18.5M [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/18.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/409267 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50530 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/51557 [00:00<?, ? examples/s]

In [22]:
dataset ,val_ds, train_ds

(DatasetDict({
     train: Dataset({
         features: ['seq', 'labels', 'labels_str', 'id'],
         num_rows: 409267
     })
     test: Dataset({
         features: ['seq', 'labels', 'labels_str', 'id'],
         num_rows: 50530
     })
     dev: Dataset({
         features: ['seq', 'labels', 'labels_str', 'id'],
         num_rows: 51557
     })
 }),
 Dataset({
     features: ['seq', 'labels', 'labels_str', 'id'],
     num_rows: 40927
 }),
 Dataset({
     features: ['seq', 'labels', 'labels_str', 'id'],
     num_rows: 368340
 }))

In [29]:
unique_labels = set(train_ds["labels_str"])
num_labels = len(unique_labels)
label2id = {label: i for i, label in enumerate(sorted(unique_labels))}
id2label = {i: label for label, i in label2id.items()}

In [23]:
unique_labels = set(train_ds["labels_str"]).shape[0]
num_labels = len(unique_labels)
label2id = {label: i for i, label in enumerate(sorted(unique_labels))}
id2label = {i: label for label, i in label2id.items()}

In [30]:
num_labels

16043

In [25]:
model_name = "facebook/esm2_t33_650M_UR50D"

from transformers import AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    problem_type="multi_label_classification",
    id2label=id2label,
    label2id=label2id,
)

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from itertools import chain

all_labels = list(set(chain(*train_ds["labels_str"])))
label2id = {label: i for i, label in enumerate(sorted(all_labels))}
id2label = {i: label for label, i in label2id.items()}
num_labels = len(label2id)

In [ ]:
!nvidia-smi

In [9]:
def preprocess(batch):
    tokens = tokenizer(
        batch["seq"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    labels = []
    for label_list in batch["labels_str"]:
        label_vec = [0.0] * num_labels  # float zeros
        for label in label_list:
            if label in label2id:
                label_vec[label2id[label]] = 1.0
        labels.append(label_vec)
    tokens["labels"] = labels
    return tokens

In [10]:
train_tok = train_ds.map(preprocess, batched=True)
val_tok = val_ds.map(preprocess, batched=True)
import torch
train_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"], dtype={"labels": torch.float})
val_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"], dtype={"labels": torch.float})

Map:   0%|          | 0/368340 [00:00<?, ? examples/s]

Map:   0%|          | 0/40927 [00:00<?, ? examples/s]

In [18]:
import datasets
train_tok = train_tok.cast_column("labels", datasets.Sequence(datasets.Value("float32")))
val_tok   = val_tok.cast_column("labels", datasets.Sequence(datasets.Value("float32")))

train_tok.set_format("torch")
val_tok.set_format("torch")

Casting the dataset:   0%|          | 0/368340 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/40927 [00:00<?, ? examples/s]

In [43]:
from datasets import DatasetDict

dataset = DatasetDict({
    "train": train_tok,
    "validation": val_tok
})

In [44]:
dataset.push_to_hub("Pingsz/mini-fold-pfam-tokenized")

Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/123 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/73.0M [00:00<?, ?B/s]

Creating parquet from Arrow format:   0%|          | 0/123 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/73.0M [00:00<?, ?B/s]

...

Uploading...:   0%|          | 0.00/73.1M [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/41 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/24.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Pingsz/mini-fold-pfam-tokenized/commit/f2f03f099d6f49f2adc1972bf57258639d542bf2', commit_message='Upload dataset', commit_description='', oid='f2f03f099d6f49f2adc1972bf57258639d542bf2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Pingsz/mini-fold-pfam-tokenized', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Pingsz/mini-fold-pfam-tokenized'), pr_revision=None, pr_num=None)

## Begin from here

In [1]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
import wandb

u=UserSecretsClient()
login(u.get_secret("HF"))
# print(u.get_secret("HF"))
wandb.login(key=u.get_secret("WB"))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sp4ss (sp4ss-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
from datasets import load_dataset

dataset = load_dataset("Pingsz/mini-fold-pfam-tokenized")
train_tok = dataset["train"]
val_tok = dataset["validation"]
import torch
train_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"], dtype={"labels": torch.float})
val_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"], dtype={"labels": torch.float})

README.md:   0%|          | 0.00/596 [00:00<?, ?B/s]

data/train-00000-of-00003.parquet:   0%|          | 0.00/73.0M [00:00<?, ?B/s]

data/train-00001-of-00003.parquet:   0%|          | 0.00/73.0M [00:00<?, ?B/s]

data/train-00002-of-00003.parquet:   0%|          | 0.00/73.1M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/24.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/368340 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40927 [00:00<?, ? examples/s]

In [3]:
import datasets
train_tok = train_tok.cast_column("labels", datasets.Sequence(datasets.Value("float16")))
val_tok   = val_tok.cast_column("labels", datasets.Sequence(datasets.Value("float16")))

train_tok.set_format("torch")
val_tok.set_format("torch")

Casting the dataset:   0%|          | 0/368340 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/40927 [00:00<?, ? examples/s]

In [4]:
dataset = load_dataset("DanielHesslow/SwissProt-Pfam")
train_ds = dataset["train"].train_test_split(test_size=0.1, seed=42)
val_ds = train_ds["test"]
train_ds = train_ds["train"]

unique_labels = set(train_ds["labels_str"]).shape[0]
num_labels = len(unique_labels)
label2id = {label: i for i, label in enumerate(sorted(unique_labels))}
id2label = {i: label for label, i in label2id.items()}

README.md:   0%|          | 0.00/494 [00:00<?, ?B/s]

dataset_infos.json: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/149M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/18.5M [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/18.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/409267 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50530 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/51557 [00:00<?, ? examples/s]

AttributeError: 'set' object has no attribute 'shape'

In [5]:
from itertools import chain

# Flatten all label lists into one list
all_labels = list(chain.from_iterable(train_ds["labels_str"]))
unique_labels = set(all_labels)
num_labels = len(unique_labels)

label2id = {label: i for i, label in enumerate(sorted(unique_labels))}
id2label = {i: label for label, i in label2id.items()}

In [10]:
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

In [8]:
model_name = "facebook/esm2_t33_650M_UR50D"
num_labels = train_tok["labels"][0].shape[0]

id2label = {i: f"label_{i}" for i in range(num_labels)}
label2id = {v: k for k, v in id2label.items()}

tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    problem_type="multi_label_classification",
    id2label=id2label,
    label2id=label2id
)

tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

2025-08-09 23:59:18.911842: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754783959.091768      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754783959.143829      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
num_labels

21

In [7]:
# model_name = "facebook/esm2_t33_650M_UR50D"

# from transformers import AutoModelForSequenceClassification
# tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False)
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=num_labels,
#     # num_labels=21,
#     problem_type="multi_label_classification",
#     id2label=id2label,
#     label2id=label2id,
# )

NameError: name 'AutoTokenizer' is not defined

In [4]:
print(type(train_tok[0]['labels']), type(val_tok[0]['labels'][0]))
len(train_tok), len(val_tok)

<class 'torch.Tensor'> <class 'torch.Tensor'>


(368340, 40927)

In [9]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
    labels = labels.astype(int)

    return {
        "f1_weighted": f1_score(labels, preds, average="weighted", zero_division=0),
        "f1_micro": f1_score(labels, preds, average="micro", zero_division=0),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
    }


In [31]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [15]:
# wandb.finish()

In [40]:
# del model1
del training_args
del trainer
import gc
gc.collect()
torch.cuda.empty_cache()

In [26]:
print(model.classifier.out_proj.out_features)  # should be 21

16043


In [34]:
# model1=model
# print(model1.classifier.out_proj.out_features)  # should be 21

21


In [35]:
# test
# model1=model
training_args = TrainingArguments(
    output_dir="./pfam-clf",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="eval_accuracy",
    metric_for_best_model="f1_micro",
    report_to="wandb",
    run_name="esm2-650M",
    # fp16=True
)
trainer = Trainer(
    model=model1,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    compute_metrics=compute_metrics,
)
trainer.train()
# del model1
# import gc
# gc.collect()
# torch.cuda.empty_cache()

Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 21.12 MiB is free. Process 7513 has 15.87 GiB memory in use. Of the allocated memory 15.55 GiB is allocated by PyTorch, and 23.31 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [12]:
model

EsmForSequenceClassification(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 1280, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-32): 33 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=1280, out_features=1280, bias=True)
              (key): Linear(in_features=1280, out_features=1280, bias=True)
              (value): Linear(in_features=1280, out_features=1280, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=1280, out_features=1280, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          )
          (inte

In [17]:
# # Smaller Subset Temporarily for Prototyping
# train_small = train_tok.select(range(10000))
# val_small = val_tok.select(range(2000))
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_small,
#     eval_dataset=val_small,
#     compute_metrics=compute_metrics,
# )
# dataloader_pin_memory=True,
# dataloader_num_workers=2,  # Tune based on Colab limits
# training_args = TrainingArguments(
#     output_dir="./pfam-clf",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     num_train_epochs=3,
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=1,
#     gradient_accumulation_steps=8,
#     save_total_limit=2,
#     load_best_model_at_end=True,
#     # metric_for_best_model="eval_accuracy",
#     metric_for_best_model="f1_micro",
#     report_to="wandb",
#     run_name="subset-esm2-650M",
#     dataloader_pin_memory=True,
#     dataloader_num_workers=2,
#     fp16=True,
#     fp16_full_eval=True,

# )
# trainer.train()

AttributeError: `AcceleratorState` object has no attribute `distributed_type`. This happens if `AcceleratorState._reset_state()` was called and an `Accelerator` or `PartialState` was not reinitialized.

In [11]:
#@title  Train 5
training_args = TrainingArguments(
    output_dir="./pfam-clf",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="eval_accuracy",
    metric_for_best_model="f1_micro",
    report_to="wandb",
    run_name="esm2-650M",
    fp16=True
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    compute_metrics=compute_metrics,
)
trainer.train()
model.push_to_hub("Pingsz/mini-fold-classifier-esm2/esm2-650M-epoch3", use_auth_token=True)
tokenizer.push_to_hub("Pingsz/mini-fold-classifier-esm2/esm2-epoch3", use_auth_token=True)

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [41]:
#@title  Train 5
training_args = TrainingArguments(
    output_dir="./pfam-clf",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="eval_accuracy",
    metric_for_best_model="f1_micro",
    report_to="wandb",
    run_name="esm2-650M"
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    compute_metrics=compute_metrics,
)
trainer.train()
model.push_to_hub("Pingsz/mini-fold-classifier-esm2/esm2-650M-epoch2", use_auth_token=True)
tokenizer.push_to_hub("Pingsz/mini-fold-classifier-esm2/esm2-epoch2", use_auth_token=True)

OutOfMemoryError: CUDA out of memory. Tried to allocate 320.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 217.12 MiB is free. Process 7513 has 15.67 GiB memory in use. Of the allocated memory 14.99 GiB is allocated by PyTorch, and 401.34 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./pfam-clf",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    report_to="wandb",
    # run_name="subset-esm2-650M",
    dataloader_pin_memory=True,
    dataloader_num_workers=2,
    fp16=True,
    fp16_full_eval=True,
)

# Define subset
train_small = train_tok.select(range(10000))
val_small = val_tok.select(range(2000))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_small,
    eval_dataset=val_small,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
model.push_to_hub("Pingsz/mini-fold-classifier-esm2/sub-esm2-650M", use_auth_token=True)
tokenizer.push_to_hub("Pingsz/mini-fold-classifier-esm2/sub-esm2-650M", use_auth_token=True)

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification", model="Pingsz/mini-fold-classifier-esm2/esm2-650M")
test_sequences = [
    "MKTFFLILALLFSAYAGDTVNSVSLTSDVK",
    "MAEPRQEFEVMVVRVDPRLCVILKRRPVEK",
    "MGSSHHHHHHSSGLVPRGSHMGSH",
    "MKADRLKAVVHVNSGQG",  # Shorter sequence
    "MGLSDGEWQLVLNVWGKVEADIPGHGQEVL"  # Hemoglobin fragment
]

for seq in test_sequences:
    result = classifier(seq)
    print(f"Sequence: {seq}\nPrediction: {result}\n")